# Net coarsened partitioning

Smoother workflow for net coarsened partitioning. Coarsened methods can handle larger networks with more complex topologies, and improve efficiency by parallelisation.

In [1]:
import numpy as np
from copy import deepcopy
from qiskit.circuit.library import QuantumVolume, QFT
from disqco.graphs.quantum_network import QuantumNetwork, linear_coupling, grid_coupling, all_to_all, random_coupling
from disqco.graphs.QC_hypergraph import QuantumCircuitHyperGraph

num_qubits = 128
num_qpus = 16 # Number of QPUs in network
qpu_capacity = int(np.ceil(num_qubits / num_qpus)) + 1 
coarsening_factor = 2  # 

# Create the quantum circuit 
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction

# circuit = QFT(num_qubits, do_swaps=False)
# circuit = QuantumVolume(num_qubits, depth=num_qubits)
circuit = cp_fraction(num_qubits, depth=num_qubits, fraction=0.5)

# Transpile to the right gate set
circuit = transpile(circuit, basis_gates=['u', 'cp'])


qpu_sizes = [qpu_capacity] * num_qpus  
connectivity = linear_coupling(num_qpus)  
# connectivity = random_coupling(num_qpus)  
# connectivity = grid_coupling(num_qpus)  
# connectivity = all_to_all(num_qpus)  # Fully connected network
# connectivity = tree_network(num_qpus)  # Tree network


initial_network = QuantumNetwork(qpu_sizes, connectivity)

network = deepcopy(initial_network)  

hypergraph = QuantumCircuitHyperGraph(circuit)


In [2]:
from disqco.parti.FM.net_coarsened_partitioning import run_full_net_coarsened_FM, check_assignment_validity


results = run_full_net_coarsened_FM(
    hypergraph=hypergraph,
    num_qubits=num_qubits,
    network=initial_network,  
    coarsening_factor=2,
    passes_per_level=10,
    use_multiprocessing=True,
    ML_internal_level_limit=100
)
final_assignment = results['best_assignment']
print(f"Final cost: {results['best_cost']}")

# Check final assignment validity
if check_assignment_validity(final_assignment, initial_network.qpu_sizes, hypergraph):
    print("✅ Final assignment is valid!")
else:
    print("Final assignment is NOT valid! Check capacity constraints and partitioning.")
    raise ValueError("Final assignment does not satisfy capacity constraints.")

Final cost: 9115
✅ Final assignment is valid!


We can also call net coarsened partitioning via the ```partitioner``` class.

In [3]:
from disqco.parti.FM.fiduccia import FiducciaMattheyses
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener

partitioner = FiducciaMattheyses(circuit, network=network)

results = partitioner.net_coarsened_partition(coarsening_factor=2,hypergraph_coarsener=HypergraphCoarsener().coarsen_recursive_subgraph_batch)

final_cost = results['best_cost']
final_assignment = results['best_assignment']

print(f"Final cost: {final_cost}")

Final cost: 9060
